# 比對EDM月報並合併

已知情況，需要將`ddd-valkyrie.edm.targetplus_edm_list_partition`的每月名單（c.名單表）  
對照`ddd-data-pipeline.eomi_synchronized.emailJobPool`（b.寄信表）  
合成 `每月EDM執行月報`(a.月報表)  
只能比對名單表的 `topic`(), `priority`, `senddate`  

對應的欄位關係如下
```
a.寄件標題 = b.subject
a.使用標題 = c.topic
a.發送日期 = b.senddate = c.senddate
a.主題模型 = c.priority(priority只取第一個\_後面的值)
a.Launchid = b.Launchid
a.寄送數量 = b.send
a.收信人數 = b.receive
a.收信比例 = b.receive_rate
a.開信人數 = b.open
a.開信比例 = b.open_rate
a.點擊EDM人數 = b.click
a.點擊比例 = b.click_rate
a.寄送失敗人數 = b.fail
a.失敗比例 = b.fail_rate
```

最後組成的檔案依照月報表的欄位排序

In [ ]:
-- 名單資料
SELECT DISTINCT topic, priority, senddate
FROM `ddd-valkyrie.edm.targetplus_edm_list_partition`
WHERE senddate >= '2024-08-01' AND topic NOT LIKE '%CR%' AND topic NOT LIKE 'EDN%'
ORDER BY senddate

In [ ]:
-- 電子報系統寄件與成效資料
select 
  Date(completetime) as senddate, 
  subject, 
  launchid,
  send,
  receive, 
  round(receive/send, 3) as receive_rate,
  open, 
  round(open/receive, 3) as open_rate, 
  click,
  round(click/open, 3) as click_rate,
  fail,
  round(fail/send, 3) as fail_rate
from (
  SELECT 
    completetime, 
    fromdisplay, 
    subject, 
    a.launchid, 
    CAST(EMAILCOUNT AS INT) AS send, 
    (CAST(EMAILCOUNT AS INT) - CAST(DISTINCTRETURNCOUNT AS INT)) AS receive,
    CAST(DISTINCTOPENCOUNT AS INT) as open, 
    CAST(DISTINCTCLICKCOUNT AS INT) as click, 
    CAST(DISTINCTRETURNCOUNT AS INT) as fail
  FROM `ddd-data-pipeline.eomi_synchronized.emailJobPool` a
  where FROMDISPLAY = '聯合報系會員特刊'
  and date_trunc(date(completetime), month) = '2023-12-01' )
  
order by senddate, launchid

In [3]:
# 確認是否安裝fuzzy
!pip show fuzzywuzzy pandas

Name: fuzzywuzzy
Version: 0.18.0
Summary: Fuzzy string matching in python
Home-page: https://github.com/seatgeek/fuzzywuzzy
Author: Adam Cohen
Author-email: adam@seatgeek.com
License: GPLv2
Location: z:\2022 BD中心\個人工作區\周遠忻\.venv\Lib\site-packages
Requires: 
Required-by: 
---
Name: pandas
Version: 2.2.2
Summary: Powerful data structures for data analysis, time series, and statistics
Home-page: https://pandas.pydata.org
Author: 
Author-email: The Pandas Development Team <pandas-dev@python.org>
License: BSD 3-Clause License

Copyright (c) 2008-2011, AQR Capital Management, LLC, Lambda Foundry, Inc. and PyData Development Team
All rights reserved.

Copyright (c) 2011-2023, Open source contributors.

Redistribution and use in source and binary forms, with or without
modification, are permitted provided that the following conditions are met:

* Redistributions of source code must retain the above copyright notice, this
  list of conditions and the following disclaimer.

* Redistributions in 

In [2]:
# 如果沒有可以安裝fuzzy
!pip install fuzzywuzzy pandas

  Using cached fuzzywuzzy-0.18.0-py2.py3-none-any.whl.metadata (4.9 kB)
Using cached fuzzywuzzy-0.18.0-py2.py3-none-any.whl (18 kB)


In [4]:
# 轉檔案編碼另存Big5，可以直接透過Excel開啟(如果有需要的話)
import pandas as pd

send_tabel_big5 = pd.read_csv('aug_寄件表.csv', encoding='utf8')
send_tabel_big5.to_csv('aug_寄件表_big5.csv', encoding='big5', index=False)
list_tabel_big5 = pd.read_csv('aug_名單表.csv', encoding='utf8')
list_tabel_big5.to_csv('aug_名單表_big5.csv', encoding='big5', index=False)

In [4]:
# V2.1_0902
import pandas as pd
from fuzzywuzzy import fuzz
from datetime import datetime, timedelta

def get_chinese_number(number):
    ranges = [
        (4600, 6050, '五千'),
        (9700, 13000, '一萬'),
        (19400, 23000, '兩萬'),
        (29100, 33000, '三萬'),
        (38800, 44000, '四萬'),
        (48800, 55000, '五萬'),
        (97500, 115000, '十萬'),
        (185000, 234000, '二十萬'),
        (244000, 282500, '二十五萬'),
        (272500, 331000, '三十萬'),
        (485000, float('inf'), '五十萬以上')
    ]
    
    for low, high, description in ranges:
        if low <= number < high:
            return description
    return '--請人工確認--'

def create_monthly_report(send_table, list_table, threshold=6, date_tolerance=0):
    # 轉換日期列
    send_table['senddate'] = pd.to_datetime(send_table['senddate'])
    list_table['senddate'] = pd.to_datetime(list_table['senddate'])
    
    # 創建空的月報表框架
    columns = ['需求單位', 'EDM類型', '寄件標題', '使用標題', '發送日期', '發送檔數', '單檔份數', '主題模型',
               '關鍵字名單編號', '其他篩選條件', 'Launchid', '寄送數量', '收信人數', '收信比例', '開信人數',
               '開信比例', '點擊EDM人數', '點擊比例', '寄送失敗人數', '失敗比例']
    rows = []
    
    # 遍歷寄件表，創建月報表行
    for _, row_b in send_table.iterrows():
        # 在名單表中尋找匹配
        matches_c = list_table[
            (list_table['senddate'] >= row_b['senddate'] - timedelta(days=date_tolerance)) &
            (list_table['senddate'] <= row_b['senddate'] + timedelta(days=date_tolerance))
        ]
        
        if not matches_c.empty:
            matches_c['similarity'] = matches_c['topic'].apply(lambda x: fuzz.ratio(x, row_b['subject']))
            best_match_c = matches_c.loc[matches_c['similarity'].idxmax()]
            
            if best_match_c['similarity'] >= threshold:
                priority_parts = best_match_c['priority'].split('_')
                main_model = priority_parts[1] if len(priority_parts) > 1 else ''
                keyword_list = priority_parts[2] if len(priority_parts) > 2 else ''
                
                new_row = {
                    '需求單位': '',  # 空白欄位
                    'EDM類型': '',  # 空白欄位
                    '寄件標題': row_b['subject'],
                    '使用標題': best_match_c['topic'],
                    '發送日期': row_b['senddate'],
                    '發送檔數': '',  # 空白欄位
                    '單檔份數': get_chinese_number(row_b['send']),  # 新增：根據send決定單檔份數
                    '主題模型': main_model,
                    '關鍵字名單編號': keyword_list,
                    '其他篩選條件': '',  # 空白欄位
                    'Launchid': row_b['launchid'],
                    '寄送數量': row_b['send'],
                    '收信人數': row_b['receive'],
                    '收信比例': row_b['receive_rate'],
                    '開信人數': row_b['open'],
                    '開信比例': row_b['open_rate'],
                    '點擊EDM人數': row_b['click'],
                    '點擊比例': row_b['click_rate'],
                    '寄送失敗人數': row_b['fail'],
                    '失敗比例': row_b['fail_rate']
                }
                rows.append(new_row)
    
    return pd.DataFrame(rows, columns=columns)

# 讀取CSV文件
send_table = pd.read_csv('aug_寄件表.csv') #這裡根據需求更換月份
list_table = pd.read_csv('aug_名單表.csv')

# 創建月報表
report_table = create_monthly_report(send_table, list_table)

# 保存結果
report_table.to_csv('aug_月報表.csv', index=False, encoding='big5') #這裡根據需求更換月份
print(f"生成的月報表行數: {len(report_table)}")
print("月報表已創建，結果已保存到腳本所在資料夾")

生成的月報表行數: 63
月報表已創建，結果已保存到 '月報表.csv'


z:\2022 BD中心\個人工作區\周遠忻\.venv\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
C:\Users\user\AppData\Local\Temp\ipykernel_14724\1779631009.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matches_c['similarity'] = matches_c['topic'].apply(lambda x: fuzz.ratio(x, row_b['subject']))
C:\Users\user\AppData\Local\Temp\ipykernel_14724\1779631009.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://panda